<a href="https://colab.research.google.com/github/ror74559/Covid-19_Valenca_2020/blob/main/covid_Valen%C3%A7a_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Casos confirmados de COVID -19 em Valença-RJ
**Fonte site da Prefeitura Municipal de Valença**
* Dados atualizados diariamente de acordo com o Boletim da Prefeitura Municipal de Valença

**Objetivo**
* Comparar o aumento do número de casos dos últimos 7 dias(última semana) com a penúltima semana.

* Construir um gráfico de corrida de barras com a evolução dos casos confirmados em cada dia, separados por mês.

# Importando dataset

In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np



In [31]:
data = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vS-Dh2cD9idWoSRvQiquQZhp4-2Yuo6oZP78u7YIKsrpXg6ag6k81mgcKOnQCXzBEcBJXZ8hslkzkYR/pub?output=csv')


# Preparando dados para utilizar o módulo python Bar Chart Race
*Copiei esse data set do site da Prefeitura, que tinha dados até o dia 21-11*

*Atualizei os dias que estavam faltando através dos boletins diários da prefeitura publicados no Facebook*

*Preparei o csv antes de importar, diretamente no Google Planilhas, alterando também o formato da data, por isso, não farei nenhuma limpeza de dados*

In [32]:
df = data.filter(['Data', 'casos confirmados'])
df.head()

,Data,casos confirmados
0,2020-05-23,79
1,2020-05-24,79
2,2020-05-25,80
3,2020-05-26,81
4,2020-05-27,82


In [33]:
# Observando os dados do dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Data               227 non-null    object
 1   casos confirmados  227 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.7+ KB


Vou mudar o tipo da coluna Data para datetime64

In [34]:
df['Data'] = df['Data'].astype('datetime64')

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Data               227 non-null    datetime64[ns]
 1   casos confirmados  227 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 3.7 KB


Modificação feita

In [36]:
#A variável 'atualização' será acrecentada na configuração do gráfico(representa a última de atualização dos dados)
ultima_data = (df['Data'][len(df['Data']) -1])
d = ultima_data.day
m = ultima_data.month
if d < 10:
  d = '0'+str(d)
if m < 10:
  m = '0'+str(m)
atualizacao = f'{d}/{m}/{ultima_data.year}'
atualizacao

'04/01/2021'

# Casos por dia
*Esse data set possui em cada linha a soma dos casos do dia mais os casos*

 *anteriores(casos acumulados) e para contruir o gráfico de barras, que será separado por mês,*
 
*primeiro precisamos do valor de  casos por dia sem somar com os dias 
anteriores*

In [37]:
#Calculando a diferença de cada linha com a linha anterior.
#Substituindo o primeiro elemento de caso_dia pelo primeiro elemento de df['casos confirmados']
caso_dia = df['casos confirmados'].diff()
print('Antes da substituição ',caso_dia[0])
caso_dia[0] = df['casos confirmados'][0]
print('Depois da substituição ',caso_dia[0])

Antes da substituição  nan
Depois da substituição  79.0


In [38]:
type(caso_dia)

pandas.core.series.Series

In [39]:
#verificando se a soma total confere com o somatório de casos
sum(caso_dia)

2223.0

Soma OK

Substituiundo os elemetos da coluna 'casos confirmados' pelos elementos da lista caso_dia

In [40]:
df['casos confirmados'] = caso_dia

In [41]:
df

,Data,casos confirmados
0,2020-05-23,79.0
1,2020-05-24,0.0
2,2020-05-25,1.0
3,2020-05-26,1.0
4,2020-05-27,1.0
...,...,...
222,2020-12-31,15.0
223,2021-01-01,22.0
224,2021-01-02,72.0
225,2021-01-03,0.0


# Comparação dos 7 dias(última semana) com a semana anterior(penúltima semana)

Essa comparação será feita com o somatório de casos confirmados de cada uma das duas últimas semanas, para verificar a evolução dos casos confirmados.

In [42]:
#importando o módulo de expressões regulares(re) para extrair da coluna 'Data', apenas a data,
# pois o formato datetime retorna a data com a hora(Timestamp) dessa forma Exemplo:
df['Data'].iloc[0]

Timestamp('2020-05-23 00:00:00')

In [43]:

import re

dados_calculo = df['casos confirmados'].tail(14) #obtendo os 14 últimos registros de casos confirmados
dados_data = df['Data'].tail(14) # obtendo os 14 últimos registros de datas


soma_ultima_semana = dados_calculo[7:].sum()
soma_penultima_semana = dados_calculo[:7].sum()

#função retorna a data do formato dd-mm-aaaa
def retorna_data(pos):
  expressao = r"\d{4}-\d{2}-\d{2}"# expressão para extrair a data com o re
  sep = '/' #variavel que guarda o caracter que irá juntar as datas
  return sep.join((re.findall(expressao,str(dados_data.iloc[pos]))[0].split('-')[::-1]))

ultima_sem_dia_in = retorna_data(7)
ultima_sem_dia_final =  retorna_data(13)
penultima_sem_dia_in =  retorna_data(0)
penultima_sem_dia_final =  retorna_data(6)


comparacao = soma_ultima_semana/soma_penultima_semana

print(f'Soma de casos da última semana ({ultima_sem_dia_in} a {ultima_sem_dia_final}): {int(soma_ultima_semana)} confirmados.')
print(f'Soma de casos da penúltima semana ({penultima_sem_dia_in} a {penultima_sem_dia_final}): {int(soma_penultima_semana)} confirmados.')
if comparacao > 1:
  aumento = ((comparacao - 1) * 100)
  print(f'Houve um aumento de {aumento:.1f}% em comparação dessa semana em relação a semana anterior.')
elif comparacao < 1:
  reducao = (1 - comparacao) * 100
  print(f'Houve uma redução de {reducao:.1f} % em comparação dessa semana em relação a semana anterior.')
else:
  print('A quantidade de casos de infectados foi igual nas duas semanas.')


Soma de casos da última semana (29/12/2020 a 04/01/2021): 304 confirmados.
Soma de casos da penúltima semana (22/12/2020 a 28/12/2020): 231 confirmados.
Houve um aumento de 31.6% em comparação dessa semana em relação a semana anterior.


# Agrupando os dados de cada mês

In [44]:
#Criando uma tupla com o nome de cada mês
meses = ('Janeiro','Fevereiro','Março','Abril','Maio','Junho','Julho','Agosto','Setembro','Outubro','Novembro','Dezembro')

In [45]:
#Renomeando o índice para o nome de cada mês da lista meses criada anteriormente acompanhada do respectivo ano
lista_nome_indice = []
for dia in df['Data']:
  mes = int (dia.month)
  indice = meses[mes - 1] + '/' + str(dia.year)[2:]
  if indice not in lista_nome_indice:
    lista_nome_indice.append(indice)
lista_nome_indice

['Maio/20',
 'Junho/20',
 'Julho/20',
 'Agosto/20',
 'Setembro/20',
 'Outubro/20',
 'Novembro/20',
 'Dezembro/20',
 'Janeiro/21']

In [46]:
#criando um data frame vazio com as colunas representando cada dia
colunas = []
for i in range(1,32):
  colunas.append(i)
df_corrida = pd.DataFrame(columns= colunas)
df_corrida

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31


In [47]:
'''
  Essa iteração agrupa os dados em listas que representam cada mês e os meses que não possuem 31 dias 
  acrescentam 0 ao final. No mês de Maio são acrescentados zeros nos dias anteriores ao inicio da primeira 
  contagem registrada.
  Após cada iteração, acrescenta cada linha ao Data Frame

'''
anos = []
for ano in df['Data']:
  if ano.year not in anos:
    anos.append(ano.year)

for a in anos:
  lista_ano = df[df['Data'].dt.year == a]
  contador = []
  for mes in lista_ano['Data']:
    if mes.month not in contador:
      contador.append(mes.month)#determina a quantidade de meses de cada ano que está sendo calculada e lembrando  que a contagem em 2020 se iniciou no mês de Maio.

  for i in range(len(contador)):
    if a == 2020:
      linha_nova = [x for x in lista_ano[lista_ano['Data'].dt.month == i + 5 ]['casos confirmados']]
    else:
       linha_nova = [x for x in df[df['Data'].dt.month == i + 1]['casos confirmados']]
    while len(linha_nova) < 31:

      if i == 0 and a == 2020:
        linha_nova.insert(0,str(i))
      else:
        linha_nova.append(0)
    df_corrida.loc[len(df_corrida)] = linha_nova

In [48]:
df_corrida

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,79.0,0.0,1.0,1.0,1.0,1.0,4.0,0.0,4.0
1,0,7,4,6,4,4,0,3,1,1,3,0,1,0,2,2,1,6,6,1,2,2,3.0,4.0,4.0,3.0,0.0,5.0,7.0,8.0,0.0
2,7,1,4,0,0,4,13,8,4,2,1,4,8,5,7,0,5,3,0,18,5,2,4.0,7.0,3.0,2.0,3.0,7.0,3.0,6.0,9.0
3,0,0,1,11,4,11,3,0,3,8,2,15,5,4,3,1,9,11,12,6,3,2,8.0,6.0,1.0,7.0,3.0,3.0,2.0,4.0,8.0
4,8,2,7,13,16,1,0,14,6,24,4,1,2,4,14,8,2,20,8,3,5,11,15.0,13.0,10.0,3.0,1.0,15.0,2.0,12.0,0.0
5,7,21,-7,3,7,1,1,7,8,0,4,1,2,25,6,6,0,3,4,2,9,12,8.0,0.0,3.0,8.0,6.0,8.0,3.0,15.0,0.0
6,1,5,13,18,15,17,0,0,3,12,3,7,2,3,18,2,16,13,8,2,10,1,18.0,26.0,8.0,4.0,12.0,10.0,0.0,26.0,0.0
7,5,18,30,14,3,2,31,45,30,25,17,1,2,31,60,34,30,33,40,20,45,23,29.0,22.0,19.0,7.0,59.0,72.0,87.0,34.0,15.0
8,22,72,0,74,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
for i in range(len(lista_nome_indice)):
  df_corrida.rename(index= ({i:lista_nome_indice[i]}), inplace = True)
df_corrida

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
Maio/20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,79.0,0.0,1.0,1.0,1.0,1.0,4.0,0.0,4.0
Junho/20,0,7,4,6,4,4,0,3,1,1,3,0,1,0,2,2,1,6,6,1,2,2,3.0,4.0,4.0,3.0,0.0,5.0,7.0,8.0,0.0
Julho/20,7,1,4,0,0,4,13,8,4,2,1,4,8,5,7,0,5,3,0,18,5,2,4.0,7.0,3.0,2.0,3.0,7.0,3.0,6.0,9.0
Agosto/20,0,0,1,11,4,11,3,0,3,8,2,15,5,4,3,1,9,11,12,6,3,2,8.0,6.0,1.0,7.0,3.0,3.0,2.0,4.0,8.0
Setembro/20,8,2,7,13,16,1,0,14,6,24,4,1,2,4,14,8,2,20,8,3,5,11,15.0,13.0,10.0,3.0,1.0,15.0,2.0,12.0,0.0
Outubro/20,7,21,-7,3,7,1,1,7,8,0,4,1,2,25,6,6,0,3,4,2,9,12,8.0,0.0,3.0,8.0,6.0,8.0,3.0,15.0,0.0
Novembro/20,1,5,13,18,15,17,0,0,3,12,3,7,2,3,18,2,16,13,8,2,10,1,18.0,26.0,8.0,4.0,12.0,10.0,0.0,26.0,0.0
Dezembro/20,5,18,30,14,3,2,31,45,30,25,17,1,2,31,60,34,30,33,40,20,45,23,29.0,22.0,19.0,7.0,59.0,72.0,87.0,34.0,15.0
Janeiro/21,22,72,0,74,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Modelando o Data Frame para utilizar o módulo Bar Chart Race

In [50]:
# Modificando o df_corrida para o formato transversal
df_corrida = df_corrida.T
df_corrida

,Maio/20,Junho/20,Julho/20,Agosto/20,Setembro/20,Outubro/20,Novembro/20,Dezembro/20,Janeiro/21
1,0,0,7,0,8,7,1,5,22
2,0,7,1,0,2,21,5,18,72
3,0,4,4,1,7,-7,13,30,0
4,0,6,0,11,13,3,18,14,74
5,0,4,0,4,16,7,15,3,0
6,0,4,4,11,1,1,17,2,0
7,0,0,13,3,0,1,0,31,0
8,0,3,8,0,14,7,0,45,0
9,0,1,4,3,6,8,3,30,0
10,0,1,2,8,24,0,12,25,0


Temos um valor negativo no dia 3 do mês de Outubro, ao verificar no dataset percebi que realmente havia um registro de redução de casos diários e deve ter sido alguma correção feita pela própria Prefeitura nesse dia.

In [51]:
df_corrida.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 1 to 31
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Maio/20      31 non-null     object
 1   Junho/20     31 non-null     object
 2   Julho/20     31 non-null     object
 3   Agosto/20    31 non-null     object
 4   Setembro/20  31 non-null     object
 5   Outubro/20   31 non-null     object
 6   Novembro/20  31 non-null     object
 7   Dezembro/20  31 non-null     object
 8   Janeiro/21   31 non-null     object
dtypes: object(9)
memory usage: 3.7+ KB


Como as colunas não foram identificadas como número temos que modificá-las.
Modificando o tipo de cada coluna para int64

In [52]:
for coluna in df_corrida.columns:
  df_corrida[coluna] = df_corrida[coluna].astype('int64')
df_corrida.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 1 to 31
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Maio/20      31 non-null     int64
 1   Junho/20     31 non-null     int64
 2   Julho/20     31 non-null     int64
 3   Agosto/20    31 non-null     int64
 4   Setembro/20  31 non-null     int64
 5   Outubro/20   31 non-null     int64
 6   Novembro/20  31 non-null     int64
 7   Dezembro/20  31 non-null     int64
 8   Janeiro/21   31 non-null     int64
dtypes: int64(9)
memory usage: 3.7 KB


Obtendo o acumulo diário igual ao dataset original, só que agora, em cada mês.

In [53]:
for mes in lista_nome_indice:
  for dia in range(2,32):
      df_corrida[mes][dia] += df_corrida[mes][dia - 1]

Renomeando o indice para ficar informativo no gráfico

In [54]:
for d in range(1,32):
  df_corrida.rename(index = ({d:'Dia ' + str (d)}), inplace = True)
df_corrida


,Maio/20,Junho/20,Julho/20,Agosto/20,Setembro/20,Outubro/20,Novembro/20,Dezembro/20,Janeiro/21
Dia 1,0,0,7,0,8,7,1,5,22
Dia 2,0,7,8,0,10,28,6,23,94
Dia 3,0,11,12,1,17,21,19,53,94
Dia 4,0,17,12,12,30,24,37,67,168
Dia 5,0,21,12,16,46,31,52,70,168
Dia 6,0,25,16,27,47,32,69,72,168
Dia 7,0,25,29,30,47,33,69,103,168
Dia 8,0,28,37,30,61,40,69,148,168
Dia 9,0,29,41,33,67,48,72,178,168
Dia 10,0,30,43,41,91,48,84,203,168


# Instalando Bar Chart Race

In [55]:
pip install bar_chart_race

# Importando e criando o Gráfico

In [56]:
import bar_chart_race as bcr

In [57]:
bcr.bar_chart_race(df = pd.DataFrame(df_corrida),
                                      orientation='h',
                                      sort='desc',
                                      n_bars=len(lista_nome_indice),
                                      fixed_order=False,
                                      fixed_max=True,
                                      steps_per_period=10,
                                      interpolate_period=False,
                                      label_bars=True,
                                      bar_size=.95,
                                      period_label={'x': .99, 'y': .30, 'ha': 'right', 'va': 'center'},
                                      period_length=1500, 
                                      period_summary_func=lambda v, r: {'x': .99, 'y': .02,
                                      's': f'Total de casos confirmados: {v.nlargest(len(lista_nome_indice)).sum():,.0f}\n\nFonte: https://www.facebook.com/PMValenca\nhttp://www.valenca.rj.gov.br               ',
                                      'ha': 'right', 'size': 7},
                                      
                                      dpi=144,
                                      cmap='dark12',
                                      title=f'Casos confirmados de COVID-19 por mês\nAtualizado em {atualizacao}\nValença-RJ',
                                      title_size='',
                                      bar_label_size=7,
                                      tick_label_size=7,
                                      shared_fontdict={'color' : '.1'},
                                      scale='linear',
                                      writer=None,
                                      fig=None,
                                      bar_kwargs={'alpha': .7},
                                      filter_column_colors=False)

# Conclusão

<p> A transmissão da COVID-19 vem aumentando outra vez no Brasil e em Valença não está sendo diferente.
</p>
<p>Além do relaxamento das medidas de isolamento social , outro fator que está colaborando para a explosão de casos no mês de Dezembro, em Valença,de acordo com a Prefeitura, é a quantidade maior de testes sendo feitos.</p>
<p>Por mais que tenhamos uma quantidade maior de testes sendo feitos, a situação é preocupante, pois os resultados só demonstram que a transmissão está acontecendo e que a quantidade de pessoas que estão por aí com o vírus, prontas para passá-lo adiante, é bem grande.</p>
<p>Além do mais, se analisarmos bem o gráfico gerado, percebemos que em momento algum houve queda no número de casos confirmados.</p>
